In [57]:
import numpy as np
import pandas as pd
from IPython.display import display, clear_output

In [58]:
class Laguerre:
    def __init__(self, T, N, beta, sigma, epsilon, max_steps, t_values):
        self.T = T
        self.N = N
        self.beta = beta
        self.sigma = sigma
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.t_values = t_values
        self.alpha = self.sigma - self.beta
    
    
    @property
    def T(self):
        return self._T
    @T.setter
    def T(self, T):
        self._T = T
        
    @property
    def N(self):
        return self._N
    @N.setter
    def N(self, N):
        self._N = N
               
    @property
    def beta(self):
        return self._beta
    @beta.setter
    def beta(self, beta):
        self._beta = beta
        
    @property
    def sigma(self):
        return self._sigma
    @sigma.setter
    def sigma(self, sigma):
        self._sigma = sigma        
        
    @property
    def epsilon(self):
        return self._epsilon
    @epsilon.setter
    def epsilon(self, epsilon):
        self._epsilon = epsilon
          
    @property
    def max_steps(self):
        return self._max_steps
    @max_steps.setter
    def max_steps(self, max_steps):
        self._max_steps = max_steps
        
    @property
    def t_values(self):
        return self._t_values
    @t_values.setter
    def t_values(self, t_values):
        self._t_values = t_values
    
    
    
        
    def laguerre(self, n, t):
        if (self.beta>=0) and (self.beta<=self.sigma):
            if n==0:
                return np.sqrt(self.sigma)*np.exp((-self.beta*t)/2)
            elif n==1:
                return np.sqrt(self.sigma)*(1-self.sigma*t)*np.exp((-self.beta*t)/2)
            else:
                l0=np.sqrt(self.sigma)*np.exp((-self.beta*t)/2)
                l1=np.sqrt(self.sigma)*(1-self.sigma*t)*np.exp((-self.beta*t)/2)
                for i in range(2, n+1):
                    li=((2*i-1-self.sigma*t)/i)*l1 - ((i-1)/i)*l0
                    l0, l1 = l1, li
                return li

        else:
            raise ValueError("Wrong data input! Beta should be in range [0;sigma]!")
        
    
    def tabulate_laguerre(self): 
        laguerre_val = {"t_val": self.t_values}  
        for n in range(self.N + 1):
            laguerre_val[f"L_{n}"] = [self.laguerre(n, t) for t in self.t_values]
        
        return pd.DataFrame(laguerre_val)
    
        
    def find_t_epsilon(self, laguerre_values):
        for index, row in laguerre_values.iterrows():
            lag_val=row.drop("t_val")
            if (lag_val.abs() < self.epsilon).all():
                return row["t_val"]
                break
        return None 
    
    
    def integrate_laguerre(self, n, f):
        steps = 1
        prev_integral = 0
        delta = float("inf")
        
        while delta > self.epsilon:
            t_vals = np.linspace(0, self.T, steps + 1)
            delta_t = self.T / steps
            
            laguerre_values = np.array([self.laguerre(n, t) for t in t_vals])
            integrand = f(t_vals) * laguerre_values * np.exp(-self.alpha * t_vals)
            integral = np.sum(integrand * delta_t) 
            
            delta = abs(integral - prev_integral)
            prev_integral = integral
            
            steps *= 2
            if steps > self.max_steps:
                raise RuntimeError("Максимальна кількість кроків перевищена. Інтеграл не збігається.")
        
        return prev_integral